In [188]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow import keras 


import matplotlib.pyplot as plt
import seaborn as sbn
from datetime import datetime

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_privacy

from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import load_model

In [189]:
seno = pd.read_excel("Seno_freq_30_amplitud_10_ruido3.xlsx")

# Funciones

In [190]:


def Abrir_Modelo(datos,fechas,modelo,ventana,prediccion,size_test):
        


    datos = datos.values.reshape(-1, 1)

    estandarizacion = MinMaxScaler().fit(datos)
    scaled_data = estandarizacion.transform(datos)


    # dividir en train, test
    X, y = [], []
    Xf,yf = [],[]

    for i in range(len(scaled_data) - ventana - prediccion):
        X.append(scaled_data[i:i+ventana])
        y.append(scaled_data[i+ventana:i+ventana+prediccion])

        Xf.append(fechas[i:i+ventana])
        yf.append(fechas[i+ventana:i+ventana+prediccion])

    X, y = np.array(X), np.array(y)
    Xf,yf = np.array(Xf),np.array(yf)



    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size_test, shuffle=False)

    fecha_X_train, fecha_X_test, fecha_y_train, fecha_y_test = train_test_split(Xf, yf, test_size=size_test, shuffle=False)


        
    y_hat = modelo.predict(X_test, verbose=1)
    y_hat = estandarizacion.inverse_transform(y_hat)

    y_test1 = y_test.reshape(-1, 1)

    y_test1 = estandarizacion.inverse_transform(y_test1)

    y_test1 = y_test1.reshape(-1,prediccion,1)


    return y_hat, y_test1,fecha_y_test,prediccion



In [191]:
def Metricas_Jd(y_hat,y_test,prediccion):
    
    predicciones_porHora_LSTM = []
    reales_porHora_LSTM = []
    for i in range(prediccion):
            
        pred = []
        for Predicciones in y_hat:
            pred.append(Predicciones[i])
            
        real = []
        for reales in y_test:
            real.append(reales[i])
            
        predicciones_porHora_LSTM.append(pred)
        reales_porHora_LSTM.append(real)
        

    MAES_LSTM = {}
    RMSE_LSTM = {}
    
    for i in range(prediccion):
        MAE_LSTM = round(mean_absolute_error(predicciones_porHora_LSTM[i],reales_porHora_LSTM[i]),2)
        MSE_LSTM = round(mean_squared_error(reales_porHora_LSTM[0],predicciones_porHora_LSTM[i]),2)  
        MAES_LSTM[i] = MAE_LSTM
        RMSE_LSTM[i] = round(np.sqrt(MSE_LSTM),2)
    
    return MAES_LSTM,RMSE_LSTM

In [192]:
# 

In [193]:
carpeta = "Archivos_Modelos"
modelos = {}
for archivo in os.listdir(carpeta):
    # Verifica que el archivo tenga la extensión .keras
    if archivo.endswith(".keras"):
        # Construye la ruta completa del archivo
        ruta_archivo = os.path.join(carpeta, archivo)
        
        # Carga el modelo .keras
        modelo = keras.models.load_model(ruta_archivo)
        modelos[archivo] = modelo


In [194]:
modelos

{'Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_100_nodos1_100_nodos2_80_Epocas_20_Batch_t_r_l.keras': <keras.src.engine.sequential.Sequential at 0x1baafe27850>,
 'Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_130_nodos1_130_nodos2_80_Epocas_20_Batch_t_r_l.keras': <keras.src.engine.sequential.Sequential at 0x1baa2d9b4d0>,
 'Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_140_nodos1_140_nodos2_80_Epocas_20_Batch_t_r_l.keras': <keras.src.engine.sequential.Sequential at 0x1baa3e72e50>,
 'Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_10_Epocas_40_Batch_t_r_lvent120.keras': <keras.src.engine.sequential.Sequential at 0x1bab4a70a90>,
 'Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_20_Epocas_40_Batch_t_r_lvent120.keras': <keras.src.engine.sequential.Sequential at 0x1bab4a88d90>,
 'Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_30_Epocas_40_Batch_t_r_lvent120.keras': <keras.src.engine.sequential.Sequential at 0x1ba

In [195]:
# Parámetros

datos = seno['valor']
ventana = 120
prediccion = 30
fechas = datos.index
nodos1 = [200,300,350]
nodos2 = [200,300,350]
paciencia = 10
epocas = 70
activacion1 = "tanh"
activacion2 = "tanh"
activacion3 = "linear"
batch = 20
size_test = 0.2
size_val = 0.2


In [196]:
Resultados_Modelos = {}
Resultado_por_modelo = {}
for i in modelos:
    Resultado_por_modelo = {}
    y_hat, y_test1,fecha_y_test,prediccion = Abrir_Modelo(datos,fechas,modelos[i],ventana,prediccion,size_test)
    print(i[:100])

    print(f"prueba {i[60:63]}")
    Resultado_por_modelo["Nodos2"] = int(i[60:63])
    Resultado_por_modelo["Nodos1"] = int(i[49:52])
    Resultado_por_modelo["Epocas"] = int(i[71:73])
    Resultado_por_modelo["batch"] = int(i[81:83])
    Resultado_por_modelo["Activación 1"] = i[-11:-10]
    Resultado_por_modelo["Activación 2"] = i[-9:-8]
    Resultado_por_modelo["Activación 3"] = i[-7:-6]
    


    y_hat, y_test1,fecha_y_test,prediccion = Abrir_Modelo(datos,fechas,modelos[i],ventana,prediccion,size_test)
    MAE_LSTM, RMSE_LSTM = Metricas_Jd(y_hat,y_test1,prediccion)
    Resultados = {}
    Resultados['MAE'] = MAE_LSTM[0]
    Resultados['RMSE'] = RMSE_LSTM[0]

    Resultado_por_modelo["MAE"] = Resultados['MAE']
    Resultado_por_modelo["RMSE"] = Resultados['RMSE']
    

    
    Resultados_Modelos[i] = Resultado_por_modelo 



dfKeras = pd.DataFrame.from_dict(Resultados_Modelos, orient='index')


6/6 [==============================] - 0s 14ms/step
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_100_nodos1_100_nodos2_80_Epocas_20_Batch_t_r_l.kera
prueba 100
6/6 [==============================] - 0s 15ms/step
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_130_nodos1_130_nodos2_80_Epocas_20_Batch_t_r_l.kera
prueba 130
6/6 [==============================] - 0s 25ms/step
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_140_nodos1_140_nodos2_80_Epocas_20_Batch_t_r_l.kera
prueba 140
6/6 [==============================] - 0s 19ms/step
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_10_Epocas_40_Batch_t_r_lvent1
prueba 150
6/6 [==============================] - 0s 19ms/step
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_20_Epocas_40_Batch_t_r_lvent1
prueba 150
6/6 [==============================] - 0s 18ms/step
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_30_Epocas_40_Batch_t_r_lvent1
prueba 150
6/6 [===========

In [197]:
dfKeras.sort_values(by="MAE", ascending= True)




,Nodos2,Nodos1,Epocas,batch,Activación 1,Activación 2,Activación 3,MAE,RMSE
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_99_Epocas_40_Batch_t_r_lvent120.keras,150,150,99,40,n,1,0,2.33,2.99
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_50_Epocas_40_Batch_t_r_lvent120.keras,150,150,50,40,n,1,0,2.33,2.98
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_80_Epocas_40_Batch_t_r_l.keras,150,150,80,40,t,r,l,2.34,3.01
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_80_Epocas_32_Batch_t_r_l.keras,150,150,80,32,t,r,l,2.34,3.03
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_80_Epocas_15_Batch_t_r_l.keras,150,150,80,15,t,r,l,2.34,2.99
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_170_nodos1_170_nodos2_80_Epocas_20_Batch_t_r_l.keras,170,170,80,20,t,r,l,2.35,3.04
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_80_Epocas_20_Batch_r_r_l.keras,150,150,80,20,r,r,l,2.35,3.05
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_80_Epocas_20_Batch_r_t_l.keras,150,150,80,20,r,t,l,2.36,3.06
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_80_Epocas_20_Batch_r_r_r.keras,150,150,80,20,r,r,r,2.36,3.01
Seno_freq_30_amplitud_10_ruido3 Sin ruido_Modelo_150_nodos1_150_nodos2_80_Epocas_20_Batch_r_t_r.keras,150,150,80,20,r,t,r,2.36,3.03


# Buscar minimo de loss en pkl

In [198]:

# Define la carpeta donde están los archivos .pkl
carpeta = "Archivos_Modelos"
modelospkl = {}

# Recorre cada archivo en la carpeta
for archivo in os.listdir(carpeta):
    # Verifica que el archivo tenga la extensión .pkl
    if archivo.endswith(".pkl"):
        # Construye la ruta completa del archivo
        ruta_archivo = os.path.join(carpeta, archivo)
        
        # Carga el modelo .pkl
        with open(ruta_archivo, 'rb') as file:
            modelopkl = pickle.load(file)
        
        # Guarda el modelo en el diccionario
        modelospkl[archivo] = modelopkl


In [199]:
# Lista para almacenar el nombre del archivo y el mínimo 'loss'
min_losses = []

for archivo, modelo in modelospkl.items():
    # Obtiene el valor mínimo de la lista en 'loss' de cada modelo
    min_loss = min(modelo['loss'])
    # Añade el archivo y su mínimo 'loss' a la lista
    min_losses.append((archivo, min_loss))

# Ordena la lista de mayor a menor por el valor mínimo de 'loss'
min_losses.sort(key=lambda x: x[1], reverse=False)

# Muestra los resultados
for archivo, min_loss in min_losses:
    print(f"{archivo}, Mínimo 'loss': {min_loss}")


Seno_freq_30_amplitud_10_ruido3 Sin ruido_history_nodos1_150_nodos2_150_epocas_80_batch_50_t_r_l.pkl, Mínimo 'loss': 0.007975438609719276
Seno_freq_30_amplitud_10_ruido3 Sin ruido_history_nodos1_150_nodos2_150_epocas_70_batch_40_t_r_lvent120.pkl, Mínimo 'loss': 0.008030146360397339
Seno_freq_30_amplitud_10_ruido3 Sin ruido_history_nodos1_150_nodos2_150_epocas_80_batch_40_t_r_l.pkl, Mínimo 'loss': 0.008049046620726585
Seno_freq_30_amplitud_10_ruido3 Sin ruido_history_nodos1_150_nodos2_150_epocas_80_batch_70_t_r_l.pkl, Mínimo 'loss': 0.008060787804424763
Seno_freq_30_amplitud_10_ruido3 Sin ruido_history_nodos1_150_nodos2_150_epocas_80_batch_20_r_r_l.pkl, Mínimo 'loss': 0.00807198230177164
Seno_freq_30_amplitud_10_ruido3 Sin ruido_history_nodos1_150_nodos2_150_epocas_90_batch_40_t_r_lvent120.pkl, Mínimo 'loss': 0.008087354712188244
Seno_freq_30_amplitud_10_ruido3 Sin ruido_history_nodos1_150_nodos2_150_epocas_80_batch_20_r_t_l.pkl, Mínimo 'loss': 0.008121443912386894
Seno_freq_30_amplitud

In [200]:
minimo = 1000
minModelo = ""
for i in modelospkl:
    if min(modelospkl[i]['loss']) < minimo:
        minimo = min(modelospkl[i]['loss'])
        minModelo = i

In [201]:
print(minModelo)
print(minimo)

Seno_freq_30_amplitud_10_ruido3 Sin ruido_history_nodos1_150_nodos2_150_epocas_80_batch_50_t_r_l.pkl
0.007975438609719276


In [202]:
modelospkl

{'Seno_freq_30_amplitud_10_ruido3 Sin ruido_history_nodos1_100_nodos2_100_epocas_80_batch_20_t_r_l.pkl': {'loss': [0.13110461831092834,
   0.049198418855667114,
   0.04124411568045616,
   0.019142357632517815,
   0.00968419574201107,
   0.009022536687552929,
   0.008643986657261848,
   0.00858999602496624,
   0.0085789505392313,
   0.008564461953938007,
   0.008556406944990158,
   0.00854917149990797,
   0.008538549765944481,
   0.00853473600000143,
   0.008533132262527943,
   0.008532058447599411,
   0.00853230431675911,
   0.0085305105894804,
   0.008524321019649506,
   0.008526981808245182,
   0.008523821830749512,
   0.008515162393450737,
   0.008515862748026848,
   0.008508840575814247,
   0.008500555530190468,
   0.008487848564982414,
   0.008483451791107655,
   0.008477616123855114,
   0.008470498956739902,
   0.008459122851490974,
   0.008453978225588799,
   0.008451116271317005,
   0.00844465009868145,
   0.008438073098659515,
   0.008428438566625118,
   0.008423964492976665,
